# Lab 2.4: LLM Observability with Arize Phoenix

In this lab, we will learn how to instrument our LLM applications for observability using **Arize Phoenix**. Observability is crucial for understanding how your LLM applications are performing, debugging issues, and evaluating quality.

We will cover:
1.  **Setup**: Installing Phoenix and launching the local UI.
2.  **Part 1: Simple LangChain Flow**: Instrumenting a basic chain.
3.  **Part 2: Advanced LangGraph Application**: Instrumenting a stateful agent workflow.

### Prerequisites
- Ensure you have your Groq API Key ready.


In [1]:
# 1. Install Dependencies
%pip install -qU arize-phoenix arize-otel openinference-instrumentation-langchain langchain langchain-groq langgraph
%pip install -q "arize[AutoEmbeddings]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 2. Setup API Keys
import getpass
import os
from arize.otel import register

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

if "ARIZE_SPACE_ID" not in os.environ:
    os.environ["ARIZE_SPACE_ID"] = getpass.getpass("Enter your Arize Space ID: ")

if "ARIZE_API_KEY" not in os.environ:
    os.environ["ARIZE_API_KEY"] = getpass.getpass("Enter your Arize API Key: ")


## 3. Launch Arize Phoenix (Optional)
You can still launch the Phoenix application locally to view traces locally, or you can view them in the Arize Cloud dashboard.

In [3]:
# Import Arize observability components
from arize.otel import register
from getpass import getpass

# Configure Arize tracing
tracer_provider = register(
    space_id=os.environ["ARIZE_SPACE_ID"],
    api_key=os.environ["ARIZE_API_KEY"],
    project_name="arize-lab-demo"
)

# Enable automatic LangChain instrumentation
from openinference.instrumentation.langchain import LangChainInstrumentor
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

print("🔭 Arize observability configured successfully!")
print("📊 All LangChain operations will now be automatically traced")

OpenTelemetry Tracing Details
|  Arize Project: arize-lab-demo
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

🔭 Arize observability configured successfully!
📊 All LangChain operations will now be automatically traced


## Part 1: Simple LangChain Flow

We will create a simple LangChain application and instrument it using `LangChainInstrumentor`.

In [4]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# Setup a simple chain
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | llm | StrOutputParser()

# Invoke the chain
print("Invoking chain...")
response = chain.invoke({"topic": "observability"})
print(f"Response: {response}")

# Check the Phoenix UI and Arize Cloud to see the trace!

d:\Projects\GenAI Labs\Jan-2026-RAG-LangGraph-labs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Invoking chain...
Response: **Why did the chicken cross the road?**  
To improve the *observability* of its journey—now every step is logged, metrics are tracked, and the trace reveals it was part of a larger *flock workflow*! 🐔📊  

*(Bonus punchline: Without observability, we’d still be wondering if it was a flaky egg or a systemic poultry issue!)*  

---  
*Logs: "We’re here!"*  
*Metrics: "We’re here too!"*  
*Traces: "And we’re everywhere in between."*  
*Together: "Welcome to your new observable system!"* 🚀


## Part 2: Advanced LangGraph Application

Now, let's look at something more complex: a LangGraph workflow. LangGraph allows for stateful, cyclic interactions, which are harder to debug without traces.

We will build a simple agent that can allow a user to chat.

In [5]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage

# --- 2. STATE DEFINITION ---
class State(TypedDict):
    # add_messages allows us to append to the conversation history
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # We can track extra metadata for tracing
    iteration_count: int

# Define LLM Node
def researcher_node(state: State):
    """Initial research node to gather info."""
    system_msg = (
        "You are an expert researcher. Provide a detailed, technical explanation "
        "of the topic requested. Focus on architecture and performance."
    )
    # We create a specific prompt for this node to show different prompt traces
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("placeholder", "{messages}")
    ])
    chain = prompt | llm
    response = chain.invoke({"messages": state["messages"]})
    return {"messages": [response], "iteration_count": 1}

def reviewer_node(state: State):
    """Reviewer node that critiques the researcher's output."""
    system_msg = (
        "You are a critical editor. Summarize the research provided "
        "into three bullet points and add a 'Grade' (A-F) based on clarity."
    )
    # This node shows how context is passed from the previous node
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("placeholder", "{messages}")
    ])
    chain = prompt | llm
    response = chain.invoke({"messages": state["messages"]})
    return {"messages": [response]}

# Build Graph
workflow = StateGraph(State)

# Add Nodes
workflow.add_node("researcher", researcher_node)
workflow.add_node("reviewer", reviewer_node)

# Define Logic
# Start -> Research -> Review -> End
workflow.add_edge(START, "researcher")
workflow.add_edge("researcher", "reviewer")
workflow.add_edge("reviewer", END)

# Compile
graph = workflow.compile()

# Invoke Graph
print("🚀 Starting Advanced LangGraph Execution...")

inputs = {
    "messages": [HumanMessage(content="Explain the importance of tracing in LLM apps.")],
    "iteration_count": 0
}

# Streaming allows us to see the trace update in real-time in Arize/Phoenix
for output in graph.stream(inputs, stream_mode="updates"):
    for node_name, node_state in output.items():
        print(f"\n--- NODE: {node_name.upper()} ---")
        last_msg = node_state["messages"][-1]
        print(f"Content: {last_msg.content[:200]}...")

print("\n✅ Execution Complete. Check your Phoenix UI to see the multi-node trace.")

# Check Phoenix UI again. You should see a trace for this execution showing the graph steps.

🚀 Starting Advanced LangGraph Execution...

--- NODE: RESEARCHER ---
Content: Tracing is a critical component in Large Language Model (LLM) applications, enabling developers to monitor, debug, and optimize complex, distributed systems. Below is a detailed technical explanation ...

--- NODE: REVIEWER ---
Content: - **Architecture & Performance Optimization**: Tracing maps distributed LLM workflows (UI, APIs, preprocessing, model inference, databases) to identify bottlenecks, optimize latency, and balance resou...

✅ Execution Complete. Check your Phoenix UI to see the multi-node trace.


## Conclusion
You have successfully set up Arize Phoenix and instrumented both a simple LangChain flow and a LangGraph application. 
Review the traces in the Phoenix UI to understand the internal execution of your LLM calls.